In [45]:
import pandas as pd
file_path = r"https://raw.githubusercontent.com/Ana-Januario/SimilarMovies_Exercise/main/u.data"

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv(file_path, sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


In [48]:
import requests
from io import StringIO

url = "https://github.com/Ana-Januario/SimilarMovies_Exercise/blob/main/u.item?raw=true"

# Baixar o conteúdo do arquivo
response = requests.get(url)
data = StringIO(response.text)

# Definir as colunas e ler o arquivo
m_cols = [
    'movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url', 
    'unknown', 'action', 'adventure', 'animation', 'children', 'comedy', 
    'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 
    'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'war', 'western'
]
movies = pd.read_csv(data, sep='|', names=m_cols, encoding='latin-1', on_bad_lines='skip', low_memory=False)

# Exibir as primeiras linhas do dataframe
print(movies.head())

   movie_id              title release_date  video_release_date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            imdb_url  unknown  action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   adventure  animation  children  ...  fantasy  film_noir  horror  musical  \
0          0          1         1  ...        0          0       0        0   


In [53]:
movies = movies[['movie_id', 'title']]

In [54]:
ratings = pd.merge(movies, ratings)
ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [55]:
#Now the amazing pivot_table function on a DataFrame will construct a user / movie rating matrix. Note how NaN indicates missing data - movies that specific users didn't rate.

movieRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
movieRatings.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
#Let's extract a Series of users who rated Star wars:
starWarsRatings = movieRatings['Star Wars (1977)']
starWarsRatings.head()

user_id
0    5.0
1    5.0
2    5.0
3    NaN
4    5.0
Name: Star Wars (1977), dtype: float64

In [57]:
#Lets use cor to compute the pairwise correlation of Star Wars' vector of user rating with every other movie! 
# After that, we'll drop any results that have no data, 
# and construct a new DataFrame of movies and their correlation score (similarity) to Star Wars:

similarMovies = movieRatings.corrwith(starWarsRatings)
similarMovies = similarMovies.dropna()
df = pd.DataFrame(similarMovies)
df.head(10)

/workspaces/SimilarMovies_Exercise/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/workspaces/SimilarMovies_Exercise/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3000: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/workspaces/SimilarMovies_Exercise/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2991: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/workspaces/SimilarMovies_Exercise/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2848: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/workspaces/SimilarMovies_Exercise/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2848: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


,0
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398
2 Days in the Valley (1996),0.066654
"20,000 Leagues Under the Sea (1954)",0.289768
2001: A Space Odyssey (1968),0.230884
"39 Steps, The (1935)",0.106453


In [58]:
#Let's sort the results by similarity score, and we should have the movies most similar to Star Wars! 
# Except... we don't. These results make no sense at all! This is why it's important to know your data 
# clearly we missed something important.
similarMovies.sort_values(ascending=False)

title
Hollow Reed (1996)                        1.0
Man of the Year (1995)                    1.0
No Escape (1994)                          1.0
Commandments (1997)                       1.0
Cosi (1996)                               1.0
                                         ... 
Swept from the Sea (1997)                -1.0
Theodore Rex (1995)                      -1.0
I Like It Like That (1994)               -1.0
Roseanna's Grave (For Roseanna) (1997)   -1.0
Frankie Starlight (1995)                 -1.0
Length: 1410, dtype: float64

In [59]:
#Our results are probably getting messed up by movies that have only been viewed by a handful of people who 
# also happened to like Star Wars. So we need to get rid of movies that were only watched by a few people 
# that are producing spurious results. Let's construct a new DataFrame that counts up how many 
# ratings exist for each movie, and also the average rating while we're at it - 
# that could also come in handy later.

import numpy as np
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
movieStats.head()

/tmp/ipykernel_32028/2074499185.py:8: FutureWarning: The provided callable <function mean at 0x7e544861d580> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})


rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

In [60]:
#Let's get rid of any movies rated by fewer than 100 people, and check the top-rated ones that are left:
popularMovies = movieStats['rating']['size'] >= 100
movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                         size      mean
title                                                  
Close Shave, A (1995)                     112  4.491071
Schindler's List (1993)                   298  4.466443
Wrong Trousers, The (1993)                118  4.466102
Casablanca (1942)                         243  4.456790
Shawshank Redemption, The (1994)          283  4.445230
Rear Window (1954)                        209  4.387560
Usual Suspects, The (1995)                267  4.385768
Star Wars (1977)                          584  4.359589
12 Angry Men (1957)                       125  4.344000
Citizen Kane (1941)                       198  4.292929
To Kill a Mockingbird (1962)              219  4.292237
One Flew Over the Cuckoo's Nest (1975)    264  4.291667
Silence of the Lambs, The (1991)          390  4.289744
North by Northwest (1959)                 179  4.284916
Godfather, The (1972)                     413  4.283293

In [62]:
#100 might still be too low, but these results look pretty good as far as "well rated movies that people 
# have heard of." Let's join this data with our original set of similar movies to Star Wars:

#df = movieStats[popularMovies].join(pd.DataFrame(similarMovies, columns=['similarity']))

# Updated for newer Pandas releases that don't allow merging between different levels; we must flatten it first now.
mappedColumnsMoviestat=movieStats[popularMovies]
mappedColumnsMoviestat.columns=[f'{i}|{j}' if j != '' else f'{i}' for i,j in mappedColumnsMoviestat.columns]
df = mappedColumnsMoviestat.join(pd.DataFrame(similarMovies, columns=['similarity']))

In [63]:
df.head()

,rating|size,rating|mean,similarity
title,,,
101 Dalmatians (1996),109,2.908257,0.211132
12 Angry Men (1957),125,4.344000,0.184289
2001: A Space Odyssey (1968),259,3.969112,0.230884
Absolute Power (1997),127,3.370079,0.085440
"Abyss, The (1989)",151,3.589404,0.203709


In [64]:
#And, sort these new results by similarity score. That's more like it!
df.sort_values(['similarity'], ascending=False)[:15]

,rating|size,rating|mean,similarity
title,,,
Star Wars (1977),584,4.359589,1.000000
"Empire Strikes Back, The (1980)",368,4.206522,0.748353
Return of the Jedi (1983),507,4.007890,0.672556
Raiders of the Lost Ark (1981),420,4.252381,0.536117
Austin Powers: International Man of Mystery (1997),130,3.246154,0.377433
"Sting, The (1973)",241,4.058091,0.367538
Indiana Jones and the Last Crusade (1989),331,3.930514,0.350107
Pinocchio (1940),101,3.673267,0.347868
"Frighteners, The (1996)",115,3.234783,0.332729


In [75]:
#Ideally we'd also filter out the movie we started from - of course Star Wars is 100% similar to itself. 
# But otherwise these results aren't bad.
#100 was an arbitrarily chosen cutoff. Try different values - what effect does it have on the end results?

mytry = movieStats['rating']['size'] >= 250
movieStats[mytry].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                         size      mean
title                                                  
Schindler's List (1993)                   298  4.466443
Shawshank Redemption, The (1994)          283  4.445230
Usual Suspects, The (1995)                267  4.385768
Star Wars (1977)                          584  4.359589
One Flew Over the Cuckoo's Nest (1975)    264  4.291667
Silence of the Lambs, The (1991)          390  4.289744
Godfather, The (1972)                     413  4.283293
Raiders of the Lost Ark (1981)            420  4.252381
Titanic (1997)                            350  4.245714
Empire Strikes Back, The (1980)           368  4.206522
Princess Bride, The (1987)                324  4.172840
Amadeus (1984)                            276  4.163043
L.A. Confidential (1997)                  297  4.161616
Fargo (1996)                              508  4.155512
Braveheart (1995)                         297  4.151515

In [76]:
mappedColumnsMoviestat=movieStats[mytry]
mappedColumnsMoviestat.columns=[f'{i}|{j}' if j != '' else f'{i}' for i,j in mappedColumnsMoviestat.columns]
df = mappedColumnsMoviestat.join(pd.DataFrame(similarMovies, columns=['similarity']))
df.sort_values(['similarity'], ascending=False)[:15]

,rating|size,rating|mean,similarity
title,,,
Star Wars (1977),584,4.359589,1.000000
"Empire Strikes Back, The (1980)",368,4.206522,0.748353
Return of the Jedi (1983),507,4.007890,0.672556
Raiders of the Lost Ark (1981),420,4.252381,0.536117
Indiana Jones and the Last Crusade (1989),331,3.930514,0.350107
L.A. Confidential (1997),297,4.161616,0.319065
E.T. the Extra-Terrestrial (1982),300,3.833333,0.303619
Back to the Future (1985),350,3.834286,0.274839
Jaws (1975),280,3.775000,0.265459
